# Cleaning



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
realEstate = pd.read_csv('Ames Real Estate Data.csv')
housePrice = pd.read_csv('Ames_HousePrice.csv', low_memory=False)
re_table = pd.read_csv('RealEstate_Table.csv')

C:\Users\hzeig\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### House Price Data

In [5]:
# DATE - standardize, combine

date_ = housePrice.filter(regex='Yr|Mo|Date')
date_

,GarageYrBlt,MoSold,YrSold
0,1939.0,3,2010
1,1984.0,2,2009
2,1930.0,11,2007
3,1940.0,5,2009
4,2001.0,11,2009
...,...,...,...
2575,1916.0,5,2009
2576,1955.0,6,2009
2577,1949.0,8,2007
2578,2000.0,7,2007


In [10]:
housePrice['GarageYrBlt'].dtypes
housePrice['GarageYrBlt'].astype('float')
pd.to_datetime(housePrice.GarageYrBlt, format = '%Y')

SyntaxError: invalid syntax (<ipython-input-10-bf16aa03a41b>, line 5)

In [ ]:
# dictionary of housePrice column meanings for quick reference
with open('housePrice_features') as hp_feat:
    hp_dict = {}
    for line in hp_feat.readlines():
        feature, description = [*line.split(':')]
        hp_dict[feature] = description

In [ ]:
hp_dict['LotFrontage']

In [ ]:
# function for viewing columns with missing values
def nas(df):
    return df.loc[:, df.isna().sum() > 0 ].isna().sum().sort_values(ascending = False)

In [ ]:
# investigate / impute column NaNs one by one
nas(housePrice)

In [ ]:
hp_dict['PoolQC']             
housePrice.PoolQC.unique()
housePrice.PoolQC = housePrice.PoolQC.fillna('none')
housePrice.groupby('PoolQC')['SalePrice'].agg(['mean', 'std', 'count'])

In [ ]:
hp_dict['MiscFeature']                                                           # drop
housePrice.groupby('MiscFeature')['SalePrice'].agg(['mean', 'std', 'count'])
housePrice = housePrice.drop('MiscFeature', axis = 1)

In [ ]:
hp_dict['Alley']
# housePrice.Alley.unique()
housePrice.Alley.value_counts()
housePrice.Alley = housePrice.Alley.fillna('none')
housePrice.groupby('Alley')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['Fence']  
# Good Privacy - GdPr
# Good Wood - GdWo
# Minimum Privacy - MnPr
# Minimum Wood - MnWw
housePrice.Fence = housePrice.Fence.fillna('none')
housePrice.groupby('Fence')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['FireplaceQu']
housePrice.FireplaceQu = housePrice.FireplaceQu.fillna('none')
housePrice.groupby('FireplaceQu')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['LotFrontage']                                 # impute using LotArea
housePrice.loc[housePrice.LotFrontage == 31]
housePrice.groupby('LotFrontage')['SalePrice'].agg(['count','mean','std'])

#### [Iterative Imputing](https://machinelearningmastery.com/iterative-imputation-for-missing-values-in-machine-learning/)

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Lot_df = housePrice[['LotFrontage','LotArea']]
imputer = IterativeImputer()
imputer.fit(Lot_df)
LotTrans = imputer.transform(Lot_df)
Lot_Imputed = pd.DataFrame(LotTrans)
housePrice['LotFrontage'] = Lot_Imputed[0]

In [ ]:
nas(housePrice)

In [ ]:
hp_dict['GarageCond']                                                       
housePrice.GarageCond = housePrice.GarageCond.fillna('none')
housePrice.groupby('GarageCond')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['GarageQual']                                             # should we try to compile garage? (is that clusters?)
housePrice.GarageQual = housePrice.GarageQual.fillna('none')              # compare relationship between rankings 
housePrice.groupby('GarageQual')['SalePrice'].agg(['count','mean','std'])           # across garage classifications

In [ ]:
hp_dict['GarageFinish']                                             
housePrice.GarageFinish = housePrice.GarageFinish.fillna('none')
housePrice.groupby('GarageFinish')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['GarageYrBlt']                                                         
housePrice.GarageYrBlt = housePrice.GarageYrBlt.fillna('none')
housePrice.groupby('GarageYrBlt')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['GarageType']                                             
housePrice.GarageType = housePrice.GarageType.fillna('none')
housePrice.groupby('GarageType')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['BsmtExposure']                                             
housePrice.BsmtExposure = housePrice.BsmtExposure.fillna('none')
housePrice.groupby('BsmtExposure')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['BsmtFinType2']                                             
housePrice.BsmtFinType2 = housePrice.BsmtFinType2.fillna('none')
housePrice.groupby('BsmtFinType2')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['BsmtFinType1']                                             
housePrice.BsmtFinType1 = housePrice.BsmtFinType1.fillna('none')
housePrice.groupby('BsmtFinType1')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['BsmtQual']                                            # does it matter if some basements (garages) don't have           
housePrice.BsmtQual = housePrice.BsmtQual.fillna('none')       # certain classifications, and are seen as 'none'
housePrice.groupby('BsmtQual')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['BsmtCond']                                             
housePrice.BsmtCond = housePrice.BsmtCond.fillna('none')
housePrice.groupby('BsmtCond')['SalePrice'].agg(['count','mean','std'])

In [ ]:
hp_dict['MasVnrType']
housePrice.MasVnrType = housePrice.MasVnrType.fillna('unknown')
housePrice.groupby('MasVnrType')['SalePrice'].agg(['count','mean','std'])

In [ ]:
# iterative imputer again, using area-related columns
hp_dict['MasVnrArea']
size_related = housePrice.filter(regex='SF|Area$')

impute = IterativeImputer(max_iter=100)
size_trans = impute.fit_transform(size_related)
size_trans = pd.DataFrame(size_trans)
housePrice.MasVnrArea = size_trans[2]
housePrice.BsmtFinSF1 = size_trans[3]
housePrice.BsmtFinSF2 = size_trans[4]
housePrice.BsmtUnfSF = size_trans[5]
housePrice.TotalBsmtSF = size_trans[6]
housePrice.GarageArea = size_trans[8]

In [ ]:
list(enumerate(size_related))

In [ ]:
nas(housePrice)

In [ ]:
hp_dict['BsmtHalfBath']
housePrice.groupby('BsmtFullBath')['SalePrice'].agg(['count','mean','std'])
housePrice[housePrice['BsmtFullBath'].isna()]

bath_df = housePrice.filter(regex='Bath$')

impute = IterativeImputer(max_iter=100)
bath_trans = impute.fit_transform(bath_df)
bath_trans = pd.DataFrame(bath_trans)
housePrice.BsmtFullBath = bath_trans[0]
housePrice.BsmtHalfBath = bath_trans[1]

In [ ]:
list(enumerate(bath_df))

In [ ]:
nas(housePrice)

In [ ]:
hp_dict['Electrical']
housePrice[housePrice['Electrical'].isna()]
housePrice.groupby('Electrical')['SalePrice'].agg(['count','mean','std'])
housePrice['Electrical'] = housePrice['Electrical'].fillna('SBrkr')

In [ ]:
hp_dict['GarageCars']
housePrice[housePrice['GarageCars'].isna()]
housePrice.groupby('GarageCars')['SalePrice'].agg(['count','mean','std'])

garage = housePrice[['GarageCars', 'GarageArea']]

impute = IterativeImputer(max_iter=100)
garage_trans = impute.fit_transform(garage)
garage_trans = pd.DataFrame(garage_trans)
housePrice.GarageCars = garage_trans[0]

In [ ]:
nas(housePrice)

In [ ]:
housePrice.to_csv('HousePrice_Clean.csv', index=False)

#### Encoding ordinals and One Hot Encoding for nominals

In [ ]:
housePrice = pd.read_csv('HousePrice_Clean.csv')

In [ ]:
hp_num = housePrice.select_dtypes(include = [np.number])
hp_cat = housePrice.select_dtypes(exclude = [np.number])
hp_cat_ord = []
hp_cat_nom = []

In [ ]:
# how many numeric and categorical values
print("numeric", hp_num.shape[1])
print("categorical", hp_cat.shape[1])

In [ ]:
housePrice.groupby('MSZoning')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('MSZoning')
hp_cat_nom

In [ ]:
housePrice.groupby('Street')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Street')
hp_cat_nom

In [ ]:
housePrice.groupby('Alley')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Alley')
hp_cat_nom

In [ ]:
housePrice.groupby('LotShape')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('LotShape')
hp_cat_nom

In [ ]:
housePrice.groupby('LandContour')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('LandContour')
hp_cat_nom

In [ ]:
housePrice.groupby('Utilities')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Utilities')
hp_cat_nom

In [ ]:
housePrice.groupby('LotConfig')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('LotConfig')
hp_cat_nom

In [ ]:
housePrice.groupby('LandSlope')['SalePrice'].agg(['count','mean','std'])  # ordinal --> create scale map dict 
hp_cat_ord.append('LandSlope')
hp_cat_ord
scale = {"Gtl":1, "Mod":2, "Sev":3}
housePrice['LandSlope_Enc'] = housePrice['LandSlope'].replace(scale)
housePrice = housePrice.drop(columns='LandSlope')

In [ ]:
housePrice.groupby('Neighborhood')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Neighborhood')
hp_cat_nom

In [ ]:
housePrice.groupby('Condition1')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Condition1')
hp_cat_nom

In [ ]:
housePrice.groupby('Condition2')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Condition2')
hp_cat_nom

In [ ]:
housePrice.groupby('BldgType')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('BldgType')
hp_cat_nom

In [ ]:
housePrice.groupby('HouseStyle')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('HouseStyle')
hp_cat_nom

In [ ]:
housePrice.groupby('RoofStyle')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('RoofStyle')
hp_cat_nom

In [ ]:
housePrice.groupby('RoofMatl')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('RoofMatl')
hp_cat_nom

In [ ]:
housePrice.groupby('Exterior1st')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Exterior1st')
hp_cat_nom

In [ ]:
housePrice.groupby('Exterior2nd')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Exterior2nd')
hp_cat_nom

In [ ]:
housePrice.groupby('MasVnrType')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('MasVnrType')
hp_cat_nom

In [ ]:
housePrice.groupby('ExterQual')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('ExterQual')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['ExterQual_Enc'] = housePrice['ExterQual'].replace(scale)
housePrice = housePrice.drop(columns='ExterQual')
hp_cat_ord

In [ ]:
housePrice.groupby('ExterCond')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('ExterCond')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['ExterCond_Enc'] = housePrice['ExterCond'].replace(scale)
housePrice = housePrice.drop(columns='ExterCond')
hp_cat_ord

In [ ]:
housePrice.groupby('Foundation')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Foundation')
hp_cat_nom

In [ ]:
housePrice.groupby('BsmtQual')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('BsmtQual')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['BsmtQual_Enc'] = housePrice['BsmtQual'].replace(scale)
housePrice = housePrice.drop(columns='BsmtQual')
hp_cat_ord

In [ ]:
housePrice.groupby('BsmtCond')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('BsmtCond')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['BsmtCond_Enc'] = housePrice['BsmtCond'].replace(scale)
housePrice = housePrice.drop(columns='BsmtCond')
hp_cat_ord

In [ ]:
housePrice.groupby('BsmtExposure')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('BsmtExposure')
scale = {"none":0, "No":1, "Mn": 2, "Av":3, "Gd":4, "Ex":5}
housePrice['BsmtExposure_Enc'] = housePrice['BsmtExposure'].replace(scale)
housePrice = housePrice.drop(columns='BsmtExposure')
hp_cat_ord

In [ ]:
housePrice.groupby('BsmtFinType1')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('BsmtFinType1')
scale = {"none":0, "Unf":1, "LwQ": 2, "Rec":3, "BLQ":4, "ALQ":5, "GLQ":6}
housePrice['BsmtFinType1_Enc'] = housePrice['BsmtFinType1'].replace(scale)
housePrice = housePrice.drop(columns='BsmtFinType1')
hp_cat_ord

In [ ]:
housePrice.groupby('BsmtFinType2')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('BsmtFinType2')
scale = {"none":0, "Unf":1, "LwQ": 2, "Rec":3, "BLQ":4, "ALQ":5, "GLQ":6}
housePrice['BsmtFinType2_Enc'] = housePrice['BsmtFinType2'].replace(scale)
housePrice = housePrice.drop(columns='BsmtFinType2')
hp_cat_ord

In [ ]:
housePrice.groupby('Heating')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Heating')
len(hp_cat_nom)

In [ ]:
housePrice.groupby('HeatingQC')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('HeatingQC')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['HeatingQC_Enc'] = housePrice['HeatingQC'].replace(scale)
housePrice = housePrice.drop(columns='HeatingQC')
hp_cat_ord

In [ ]:
housePrice.groupby('CentralAir')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('CentralAir')
len(hp_cat_nom)

In [ ]:
housePrice.groupby('Electrical')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('Electrical')
len(hp_cat_nom)

In [ ]:
housePrice.groupby('KitchenQual')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('KitchenQual')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['KitchenQual_Enc'] = housePrice['KitchenQual'].replace(scale)
housePrice = housePrice.drop(columns='KitchenQual')
hp_cat_ord

In [ ]:
housePrice.groupby('Functional')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('Functional')
scale = {"Typ":0, "Min1":1, "Min2": 2, "Mod":3, "Maj1":4, "Maj2":5, "Sal":6}
housePrice['Functional_Enc'] = housePrice['Functional'].replace(scale)
housePrice = housePrice.drop(columns='Functional')
hp_cat_ord

In [ ]:
housePrice.groupby('FireplaceQu')['SalePrice'].agg(['count','mean','std'])
hp_cat_ord.append('FireplaceQu')
scale = {"none":0, "TA":1, "Po": 2, "Fa":3, "Gd":4, "Ex":5}
housePrice['FireplaceQu_Enc'] = housePrice['FireplaceQu'].replace(scale)
housePrice = housePrice.drop(columns='FireplaceQu')
hp_cat_ord

In [ ]:
housePrice.groupby('GarageType')['SalePrice'].agg(['count','mean','std'])
hp_cat_nom.append('GarageType')
len(hp_cat_nom)

In [ ]:
housePrice.groupby('GarageYrBlt')['SalePrice'].agg(['count','mean','std'])
# hp_cat_nom.append('Heating')
# len(hp_cat_nom)

In [ ]:
hp_cat.columns

In [ ]:
# Scale Maps for ordinal values


EQscale = {}

### Real Estate Data

In [ ]:
realEstate.shape

In [ ]:
nas(realEstate)

In [ ]:
# function to look up definitions of realEstate features
def table(feature):
    return re_table.loc[re_table['Field Name'] == feature]

In [ ]:
table('Rcrd_Mo')

In [ ]:
re_num = realEstate.select_dtypes(include=[np.number])
re_cat = realEstate.select_dtypes(exclude=[np.number])

In [ ]:
 # DATE - dropping
from datetime import date
                                                              # after seeing that date columns have too many
date_df = realEstate.filter(regex='Yr|Mo|Date')               # missing values and are complicated to combine, 
date_df.loc[~date_df['YrSold_YYYY'].isna()]                   # decided to drop and use date from HP
# date_comp = date_df.loc[date_df['Rcrd_Yr'] != date_df['Inst1_Yr']]
# date_comp.loc[~date_df['Rcrd_Yr'].isna()]
# date_df.loc[date_df['Date'] != '10-Jul-20']

date_df.columns
realEstate = realEstate.drop(date_df.columns, axis=1)
# realEstate['day'] = 1
# pd.to_datetime(realEstate[['Rcrd_Yr', 'Rcrd_Mo', 'day']].stack()).unstack
# pd.to_datetime(dte.stack()).unstack()
              
# pd.to_datetime(assign(year, month = realEstate[['Rcrd_Yr','Rcrd_Mo']]).assign(DAY=1))

In [ ]:
##  examining and dropping unnecessary features

table('Inst1_Mo')

In [ ]:
re_num.isna().sum().sort_values(ascending=False)

In [ ]:
re_cat.isna().sum().sort_values(ascending=False)